In [1]:
!pip install git+https://github.com/kernelmachine/allennlp.git@4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
!pip install transformers==2.6.0
!pip install pytorch-transformers==1.2.0
!git clone https://github.com/allenai/dont-stop-pretraining

  Cloning https://github.com/kernelmachine/allennlp.git (to revision 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7) to /tmp/pip-req-build-zsbe390q
  Running command git clone -q https://github.com/kernelmachine/allennlp.git /tmp/pip-req-build-zsbe390q
  Running command git checkout -q 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 102kB 5.2MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 30.9MB 94kB/s 
     |████████████████████████████████| 163kB 67.9MB/s 
     |████████████████████████████████| 317kB 61.4MB/s 
     |████████████████████████████████| 133kB 53.4MB/s 
     |████████████████████████████████| 266kB 58.7MB/s 
     |████████████████████████████████| 245kB 58.1MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 


In [2]:
%cd /content/dont-stop-pretraining
!pwd

/content/dont-stop-pretraining
/content/dont-stop-pretraining


In [3]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

!nvidia-smi

cuda
Tue Nov 10 03:44:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [4]:
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/chemprot/train.jsonl
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/chemprot/dev.jsonl
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/chemprot/test.jsonl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1094k  100 1094k    0     0  4277k      0 --:--:-- --:--:-- --:--:-- 4277k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  662k  100  662k    0     0  3068k      0 --:--:-- --:--:-- --:--:-- 3068k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  937k  100  937k    0     0  4712k      0 --:--:-- --:--:-- --:--:-- 4712k


In [5]:
import pandas as pd
import json
data = []
with open("train.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("train.txt", header=False, index=False)
with open("dev.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("dev.txt", header=False, index=False)

In [6]:
!ls

ADAPTIVE_PRETRAINING.md  dont_stop_pretraining	README.md     test.jsonl
DATA_SELECTION.md	 environments		scatter.pdf   training_config
dev.jsonl		 environment.yml	scripts       train.jsonl
dev.txt			 mlm_study		search_space  train.txt


In [ ]:
!python -m scripts.run_language_modeling \
                --train_data_file train.txt \
                --line_by_line \
                --output_dir roberta-tapt \
                --model_type roberta-base \
                --tokenizer_name roberta-base \
                --mlm \
                --per_gpu_train_batch_size 16 \
                --gradient_accumulation_steps 16  \
                --model_name_or_path roberta-base \
                --eval_data_file dev.txt \
                --do_eval \
                --evaluate_during_training  \
                --do_train \
                --num_train_epochs 100  \
                --learning_rate 0.0001 \
                --logging_steps 50

2020-11-10 03:50:52.898902: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/10/2020 03:50:54 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/10/2020 03:50:55 - INFO - filelock -   Lock 139707746117896 acquired on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
11/10/2020 03:50:55 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpckc2v4gs
Downloading: 100% 481/481 [00:00<00:00, 409kB/s]
11/10/2020 03:50:55 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json in cache at /root/.cache/torch/transfo